# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [41]:

import os
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import joblib

from sklearn.linear_model import LogisticRegression

import argparse

import os

import numpy as np

from sklearn.metrics import mean_squared_error

import joblib

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

import pandas as pd

from azureml.core.run import Run

from azureml.data.dataset_factory import TabularDatasetFactory


In [40]:
ws = Workspace.from_config()
experiment_name = 'detection'

experiment=Experiment(workspace=ws, name=experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136741
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-136741


In [42]:
amlcompute_cluster_name = "cpu-clusters"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded........................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [55]:
 # Create TabularDataset using TabularDatasetFactory
    # Data is located at:

data_path = "https://raw.githubusercontent.com/Tekhunt/ML-Project/master/diabetes.csv"

data =TabularDatasetFactory.from_delimited_files(path= data_path)

In [56]:
data.to_pandas_dataframe().head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [57]:

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import os

In [58]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
        "--max_iter": choice(50, 100, 150, 200, 250, 500)
    }
)

#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(source_directory = experiment_folder,
                 #entry_script = 'train.py',
                 #compute_target = cpu_cluster)
estimator =  SKLearn(source_directory='./', 
                entry_script='train.py', compute_target=aml_compute)


hyperdrive_run_config =HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                    primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    policy=policy,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    estimator=estimator
                                   )

In [59]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
#hyperdrive_run.wait_for_completion(show_output=True)

In [60]:
#hyperdrive_run.wait_for_completion(show_output=True)




RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4547f7f0-68c6-45d5-819b-6a8a10b00ef3
Web View: https://ml.azure.com/experiments/detection/runs/HD_4547f7f0-68c6-45d5-819b-6a8a10b00ef3?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-136741/workspaces/quick-starts-ws-136741

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-31T18:32:06.310800][API][INFO]Experiment created<END>\n"<START>[2021-01-31T18:32:07.3179525Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-31T18:32:07.328621][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-31T18:32:07.651748][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_4547f7f0-68c6-45d5-819b-6a8a10b00ef3
Web View: https://ml.azure.com/experiments/detection/runs/HD_4547f7f0-68c6-45d5-819b-6a8a10b00ef3?wsid=/subscriptions/f9d5a085-54dc-421

{'runId': 'HD_4547f7f0-68c6-45d5-819b-6a8a10b00ef3',
 'target': 'cpu-clusters',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-31T18:32:06.109138Z',
 'endTimeUtc': '2021-01-31T18:40:48.76723Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "Ac

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# Visualize hyperparameter tuning runs
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
run_file_names = best_run.get_file_names()
print(parameter_values)
print(/n)
print(run_file_names)
print(/n)
print(best_run_metrics)



In [ ]:
#print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['accuracy'])
#print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

In [ ]:

model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')

In [ ]:
#TODO: Save the best model
#Save and register the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service